# Peak Load Forecasting

### import

In [1]:
import csv
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
from math import sqrt
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

### load data

In [2]:
max_peak  = []         # 台電預估之 淨尖峰供電能力
peak_load = []         # 尖峰負載
nxweek_max_peak  = [] 
nxweek_peak_load = [] 

# Read from csv files
with open("power_dataset.csv", newline='') as csvfile: # 台灣電力公司_過去電力供需資訊
    rows = csv.DictReader(csvfile)
    for row in rows:
        # 將'淨尖峰供電能力'和'尖峰負載'這兩欄的資料分別存入 max_peak 和 peak_load 
        max_peak.append (float(row['max_peak']))
        peak_load.append(float(row['peak_load']))

with open("next_week.csv", newline='') as csvfile:     # 台灣電力公司_未來一週電力供需預測
    rows = csv.DictReader(csvfile)
    for row in rows:
        nxweek_max_peak.append (float(row['max_peak'])*10) # 單位換算
        nxweek_peak_load.append(float(row['peak_load'])*10)

# reshape 成 svm 接受的格式
X_train = np.array(max_peak ).reshape(-1,1)
y_train = np.array(peak_load).reshape(-1,1)
X_test  = np.array(nxweek_max_peak).reshape(-1,1)
y_test  = np.array(nxweek_peak_load).reshape(-1,1)

### training

In [3]:
# 建立 svc model 並使用 '淨尖峰供電能力'和'尖峰負載'這兩個features 進行訓練
model = SVC(kernel='poly' ,gamma='scale', degree=9, C=1.0, max_iter=20000).fit(X_train, y_train)
# 使用台電預測的下週 '淨尖峰供電能力' 進行預測
nxweek_pred = model.predict(X_test)
# 用 RMSE 進行評估
result = sqrt(mean_squared_error(y_test, nxweek_pred))

### output

In [4]:
# 輸出 submission.csv
with open('submission.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['date','peak_load(MW)'])
    for i in range(20190402,20190409):
        writer.writerow([i, int(nxweek_pred[i-20190402])])

# 印出 RMSE 和 svc model 之 score
print("predicted RMSE: {}".format(result))
print("score: {}".format(model.score(X_train,y_train)))

predicted RMSE: 165.92123776918115
score: 0.7427122940430925
